# Executing SQL from python

A quick overview of how to query the database with SQL via python

<u><b>Use cases:</u></b>
- Automate the execution of many different queries/scripts in some order
- Read in many different tables from database and other sources, do complex data wrangling in python, then easily import into database
- Can be called from PowerBI
- If no priveledges to create tables, can store them locally and use this code to integrate them with database tables (e.g. joins)
<br><br>**...and more...**

---

##  Create connection to database

### Imports

In [1]:
import pyodbc

### Details 

We have to give `server` and `database`

In [2]:
# Server address
server = 'server123'

# Database within server
database = 'mydb'

### Create connection 

Using these details + any other credentials, we create an authenticated connection to the database

In [ ]:
# Connection details, in special format
conn_string = 'DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes'

In [ ]:
# Connection object
conn = pyodbc.connect(conn_string)

### Create cursor 

This is a chunk of memory on your laptop. Anytime you execute a query, any returned results get stored in the cursor, so you can access it. 

Think of it like the "Results" pane from SSMS.

In [ ]:
cursor = conn.cursor()

---

## Execute a generic query 

Any valid SQL code can be run directly from this code script.

So this includes SELECT queries, INSERT statements, EXEC of stored proc etc.

### Define query string 

The actual query to be run

In [3]:
query = '''
    SELECT * 
    FROM dbo.blah 
    WHERE x > 5
'''

### Execute

We use the cursor to execute the query. Essentially
- The cursor sends the query to the server
- The query is executed server-side
- The results are sent back to the cursor

In [ ]:
cursor.execute(query)

### (Optional) Commit 

You only run `commit` on the **connection** if the query changes the database (e.g. INSERT statements).

Furthermore, you can set this code up so that you only commit if the `execute` was successful

In [ ]:
conn.commit()

### Close cursor and connection 

Make sure you close your cursor (i.e. free up that memory) and connection (i.e. so having got a constant live but unused connection)

In [ ]:
cursor.close()
conn.close()

---

## Execute multiple generic queries in a specific order

You simply write the `execute` commands in the order you want

**N.B.** You can put all the of the above code in the same cell. If 1 step fails, the program terminates at that point

In [ ]:
query_1 = '''
    IF OBJECT_ID('dbo.Test', 'U') IS NOT NULL
        DROP TABLE dbo.Test;
'''

query_2 = '''
CREATE TABLE dbo.Test (
    id INT PRIMARY KEY,
    test1 VARCHAR(MAX)
);
'''

query_3 = '''
    INSERT INTO dbo.Test(id, test1) VALUES
        (1, 'a'),
        (2, 'b');
'''

conn = pyodbc.connect(conn_string)
cursor = conn.cursor()
cursor.execute(query_1)
cursor.execute(query_2)
cursor.execute(query_3)
conn.commit()
cursor.close()
conn.close()

---

### Use results of SELECT query 

You can get the results of a SELECT query returned directly to a special tabular form in python called a DataFrame.

The power of this is that you can transform and model data how you want in python.

### Create connection 

In [ ]:
conn = pyodbc.connect(conn_string)

### Create SELECT query string 

In [6]:
query = '''
    SELECT test1, count(*) AS cnt 
    FROM dbo.blah 
    GROUP BY test1
    ORDER BY cnt DESC;
'''

**N.B.** NO cursor needed, as this is created automatically for us in next step

### Read results into DataFrame

In [ ]:
# Import required library
import pandas as pd

# Execute query, return results to DataFrame table
df = pd.read_sql(query, conn)

df

*If this were a real query, the results would be displayed here in nice tabular form*

---

### Insert DataFrame into database table 

You can insert the final DataFrame into a database table directly.

### Create connection (we've been using whole time) 

In [ ]:
conn = pyodbc.connect(conn_string)

### (Optional) Create table  

In [ ]:
schema_name = 'dbo'
table_name = 'Test'
query = '''
CREATE TABLE '''+schema_name+'.'+table_name+''' (
    id INT PRIMARY KEY,
    test1 VARCHAR(MAX)
);
'''

cursor = conn.cursor()
cursor.execute(query)
conn.commit()
cursor.close()
conn.close()

### Create new type of connection called 'engine'

To make life easier, it is necessary to reformulate our connection so that the next step works.

It is commonly called an `engine`, rather than `conn`

In [14]:
# Import
from sqlalchemy import create_engine

# Create 'engine'
eng_string = 'mssql+pyodbc://'+server+'/'+database+'?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server'
engine = create_engine(eng_string)

### Insert into this table 

You need to give this one command the `engine`, `table_name` and `schema_name`.

Optionally, you can give it:
-  `if_exists` = `'fail'`, if want code to fail if table exists; `'replace'`, if want to truncate table and insert only our new data; or `'append'` to append new data to existing data
- `index` = `True`, if want DataFrame's auto-generated unique row ID as a column (e.g. as the primary key); otherwise `False`
- `index_label` = name of column that will hold these newly inserted DataFrame indices

In [ ]:
df.to_sql(
    name=table_name,
    con=engine,
    schema=schema_name,
    if_exists='append',
    index=True,
    index_label='id'
)

### Close `engine` 

This is called `dispose`

In [15]:
engine.dispose()